In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mu = 1.2
alpha = .6
beta = 1.
T = 5.1
seed = 560
# seed += 1
print("Seed =", seed)
resolution = 100

np.random.seed(seed)

def phi(t):
    return alpha * np.exp(-beta * t)

def lamb(mu, phi, t, tau):
    return mu + sum([phi(t-tk) for tk in tau])

events = []
events_attempt = []
lambda_bar_list = []
D_list = []
accepted = []
s = 0
while s < T:
    lambda_bar = lamb(mu, phi, s, events)
    w = np.random.exponential(1/lambda_bar)
    s += w
    D = np.random.uniform(0,lambda_bar)
    if D <= lamb(mu, phi, s, events):
        events.append(s)
        accepted.append(True)
    else:
        accepted.append(False)
    events_attempt.append(s)
    D_list.append(D)
    lambda_bar_list.append(lambda_bar)
if events[-1] > T:
    events = events[:-1]
events_attempt = events_attempt[:-1]
D_list = D_list[:-1]
accepted = accepted[:-1]
rejected = [not a for a in accepted]

x = np.array([0, events[0]])
intensity = np.array([mu, mu])
pointsT = events + [T]
for i in range(0, len(pointsT)-1):
    p1, p2 = pointsT[i:i+2]
    local_x = np.linspace(p1, p2, resolution)
    local_intensity = [lamb(mu, phi, xi, pointsT[:i+1]) for xi in local_x]
    x = np.append(x, local_x)
    intensity = np.append(intensity, local_intensity)

fig, ax = plt.subplots(figsize=(20,6))
ax.set_xlim(0, T)
ax.set_ylim(0, max(intensity)+1)
lines = plt.plot(x, intensity, color="black", label="Intensity $\lambda(t)$")
ax.hlines(lambda_bar_list, [0]+events_attempt, events_attempt+[T], color="blue", linestyles="dashed", label=r"Maximum intensity $\bar\lambda$")
ax.scatter(np.array(events_attempt)[accepted], np.array(D_list)[accepted], s=50, color="green", label=r"$\mathcal{U}(0,\bar\lambda)$ (accepted)", zorder=10)
ax.scatter(np.array(events_attempt)[rejected], np.array(D_list)[rejected], s=50, color="red", label=r"$\mathcal{U}(0,\bar\lambda)$ (rejected)", zorder=10)
ax.vlines(events, *ax.get_ylim(), color="darkgray", linestyles="dashed", linewidth=1.1, alpha=1, label = r"Accepted event: $\mathcal{U}(0,\bar\lambda) \leq \lambda(t)$")
ax.vlines(np.array(events_attempt)[rejected], *ax.get_ylim(), color="gray", linestyles="dotted", linewidth=.6, alpha=1, label=r"Rejected event: $\mathcal{U}(0,\bar\lambda) > \lambda(t)$")
for (e1, e2) in zip([0]+events_attempt, events_attempt+[T]):
    ax.annotate("",
                xy=(e1, max(intensity)+.5), xycoords='data',
                xytext=(e2, max(intensity)+.5), textcoords='data',
                arrowprops=dict(arrowstyle="<->", connectionstyle="arc3", color='gray', lw=1.5),
                )
    ax.text((e1+e2)/2, max(intensity)+.6, r"$\mathcal{E}(\bar\lambda)$", ha="center", va="center")
ax.legend()
ax.set_title("Thinning algorithm")
ax.set_xlabel("Time")
ax.set_ylabel("Intensity")
fig.savefig("./figures/thinning_algorithm.pdf", bbox_inches="tight")
plt.show()